# Importation des bibliothèques

In [1]:
import pandas as pd
import sqlite3
import os
import warnings
warnings.filterwarnings("ignore")

# Connexion à la base de données brutes

In [2]:
connect = sqlite3.connect("raw-database.db")
cursor = connect.cursor()

# Données de matching des matchs Skill Corner / Stats Bomb

In [3]:
# Lecture de l'unique fichier pour ces données
matching_matches = pd.read_json("Projet_centres_data/matching_matches.json")

In [4]:
# Aperçu des données
matching_matches.head(2)

,statsbomb_id,skillcorner_id
0,3894366.0,1547880
1,3894367.0,1547881


Nous allons renommer les colonnes de ce dataframe pour leur donné un nom plus cohérent

In [5]:
# Dictionnaire permettant de renommer les colonnes souhaitées
dico_rename = {"statsbomb_id" : "match_id_SB", "skillcorner_id" : "match_id_SKC"}
matching_matches.rename(dico_rename, axis = 1, inplace = True)

In [6]:
# Ecriture des données importées dans une table de la BDD
matching_matches.to_sql("matching_matches", con = connect, if_exists = "replace", index = False)

306

# Données de matching des joueurs Skill Corner / Stats Bomb

In [7]:
# Lecture de l'unique fichier pour ces données
matching_players = pd.read_json("Projet_centres_data/matching_players.json")

In [8]:
# Aperçu des données
matching_players.head(2)

,birthday,trackable_object,gender,statsbomb_id,skillcorner_id
0,2006-04-05,820104,male,307251.0,818541
1,1997-11-28,39010,male,66886.0,37889


Nous allons renommer les colonnes de ce dataframe pour leur donné un nom plus cohérent

In [9]:
# Dictionnaire permettant de renommer les colonnes souhaitées
dico_rename = {"statsbomb_id" : "player_id_SB", "skillcorner_id" : "player_id_SKC"}
matching_players.rename(dico_rename, axis = 1, inplace = True)

In [10]:
# Ecriture des données importées dans une table de la BDD
matching_players.to_sql("matching_players", con = connect, if_exists = "replace", index = False)

639

# Données de matching des équipes Skill Corner / Stats Bomb

In [11]:
# Lecture de l'unique fichier pour ces données
matching_teams = pd.read_json("Projet_centres_data/matching_teams.json")

In [12]:
# Aperçu des données
matching_teams.head(2)

,statsbomb_id,skillcorner_id
0,168,85
1,144,66


Nous allons renommer les colonnes de ce dataframe pour leur donné un nom plus cohérent

In [13]:
# Dictionnaire permettant de renommer les colonnes souhaitées
dico_rename = {"statsbomb_id" : "team_id_SB", "skillcorner_id" : "team_id_SKC"}
matching_teams.rename(dico_rename, axis = 1, inplace = True)

In [14]:
# Ecriture des données importées dans une table de la BDD
matching_teams.to_sql("matching_teams", con = connect, if_exists = "replace", index = False)

18

# Informations Stats Bomb pour les matches disponibles

In [15]:
# Lecture de l'unique fichier pour ces données
SB_matches = pd.read_json("Projet_centres_data/SB_matches.json")

In [16]:
# Aperçu des données
SB_matches.head(2)

,match_id,match_date,kick_off,competition,season,home_team_id,away_team_id,home_score,away_score,attendance,...,match_status,match_status_360,last_updated,last_updated_360,match_week,competition_stage,referee,data_version,shot_fidelity_version,xy_fidelity_version
0,3894290,2024-03-08,22:00:00.000,France - Ligue 1,2023/2024,136,139,1.0,2.0,21819.0,...,available,available,2024-09-18T14:30:45.585468,2024-09-18T14:33:21.527901,25,Regular Season,Ruddy Buquet,1.1.0,2,2
1,3894367,2024-05-19,22:00:00.000,France - Ligue 1,2023/2024,165,152,5.0,0.0,16176.0,...,available,available,2024-08-07T21:15:56.762493,2024-08-07T21:17:38.580126,34,Regular Season,Clément Turpin,1.1.0,2,2


In [17]:
# Renommage des colonnes pour indiquer le fait que les id sont des id Stats Bomb
dico_rename = {"match_id" : "match_id_SB", "home_team_id" : "home_team_id_SB", "away_team_id" : "away_team_id_SB"}
SB_matches.rename(dico_rename, axis = 1, inplace = True)

In [18]:
# Ecriture des données importées dans une table de la BDD
SB_matches.to_sql("SB_matches", con = connect, if_exists = "replace", index = False)

307

# Données events de Stats Bomb

Pour ces données, nous avons un fichier par match, il faut donc importer une liste de fichiers (situés dans le dossier SB_events).  
Pour ce faire, nous allons ouvrir un par un ces fichiers et les concaténer afin d'obtenir un unique jeu de données comprenant l'ensemble des fichiers.  


In [19]:
# Liste des noms des fichiers à ouvrir
liste_fichier_events = os.listdir("Projet_centres_data/SB_events")

In [20]:
# Commande qui permet de concaténer l'ensemble des fichiers
# La boucle for à l'intérieur des [] permet de parcourir la liste des noms des fichiers, et d'ouvrir un fichier un chaque itération
events = pd.concat([pd.read_json("Projet_centres_data/SB_events/" + fichier_event) for fichier_event in liste_fichier_events])

Nous allons maintenant filtrer ces données afin de :  
- Garder uniquement les données dont nous nous servirons. En effet, ces données contiennent 108 variables(colonnes) de base, or elles ne nous sont pas toutes utiles, nous allons donc garder seulement celles qui nous sont utiles afin de réduire la taille des données.  
- Renommer certaines colonnes afin de faciliter notre utilisation de ces dernières.  
- Convertir les colonnes contenant des données stockées sous forme de liste en plusieurs colonnes comprenant chacune un élément de la liste.  
En effet, les bases de données SQLite ne supportent pas les données de type liste, dictionnaire, dataframe etc.

In [21]:
# Liste contenant les noms de colonnes que nous souhaitons garder
colonnes = ["id", "shot_type", "shot_outcome", "type", "match_id", "period", "possession", "location", "pass_cross", "pass_type", "index",
            "pass_end_location", "minute", "shot_end_location", "pass_body_part", "player_id", "team_id"]
# Commande pour garder uniquement colonnes souhaitées
events = events[colonnes]

In [22]:
# Aperçu des données
events.head(2)

,id,shot_type,shot_outcome,type,match_id,period,possession,location,pass_cross,pass_type,index,pass_end_location,minute,shot_end_location,pass_body_part,player_id,team_id
0,02591dfb-4789-442f-ae99-53ac3f3aca81,None,None,Starting XI,3894037,1,1,None,NaN,None,1,None,0,None,None,NaN,147
1,a5620f61-b542-46c8-b0b4-8e2daf267e84,None,None,Starting XI,3894037,1,1,None,NaN,None,2,None,0,None,None,NaN,156


In [23]:
# Renommage des colonnes
# Dictionnaire permettant de renommer les colonnes souhaitées
dico_rename = {"index" : "index_event", "id" : "event_id", "match_id" : "match_id_SB", "player_id" : "player_id_SB", "team_id" : "team_id_SB"}
events.rename(dico_rename, axis = 1, inplace = True)

In [24]:
# La colonne "index_event" va nous permettre, pour chaque match, de trier les events par ordre chronologique
events.sort_values(by = ["match_id_SB", "index_event"], inplace = True)

# De plus, nous utilisons la commande reset_index afin d'obtenir un unique identifiant pour chaque evenement de l'ensemble des données concaténées.
# En effet, les index du dataframe actuel ne sont pas uniques car on a effectué une concaténantion de plusieurs dataframes qui avaient des
# index en communs.
events.reset_index(drop = True, inplace = True)

In [25]:
# Les données de la colonne "location" sont des listes de 3 coordonnées, nous allons donc "éclater" cette colonne en 3 colonnes :
# "x_loc", "y_loc" et "z_loc".
# Pour ce faire, nous extrayons d'abord la colonne du dataframe, puis nous créons un autre dataframe comprenant les 3 colonnes des coordonnées
events_loc = events.pop("location").dropna()
events_loc = pd.DataFrame(events_loc.tolist(), index = events_loc.index, columns = ["x_loc", "y_loc", "z_loc"])

# De même pour la colonne "pass_end_location", sauf que celle-ci ne contient que 2 coordonnées
events_pass_loc = events.pop("pass_end_location").dropna()
events_pass_loc = pd.DataFrame(events_pass_loc.tolist(), index = events_pass_loc.index, columns = ["x_pass", "y_pass"])

# De même pour la colonne "shot_end_location" qui comprend 3 coordonnées
events_shot_loc = events.pop("shot_end_location").dropna()
events_shot_loc = pd.DataFrame(events_shot_loc.tolist(), index = events_shot_loc.index, columns = ["x_shot", "y_shot", "z_shot"])

In [26]:
# Il nous reste a concaténer les 3 dataframes créés précédemment avec le dataframe initial.
# Nous concaténons ces dataframes horizontalement (nous les ajoutons à droite du dataframe initial)
events = pd.concat([events, events_loc, events_pass_loc, events_shot_loc], axis = 1)

In [27]:
# Aperçu des données
events.head(2)

,event_id,shot_type,shot_outcome,type,match_id_SB,period,possession,pass_cross,pass_type,index_event,...,player_id_SB,team_id_SB,x_loc,y_loc,z_loc,x_pass,y_pass,x_shot,y_shot,z_shot
0,02591dfb-4789-442f-ae99-53ac3f3aca81,None,None,Starting XI,3894037,1,1,NaN,None,1,...,NaN,147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a5620f61-b542-46c8-b0b4-8e2daf267e84,None,None,Starting XI,3894037,1,1,NaN,None,2,...,NaN,156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# Ecriture des données importées dans une table de la BDD
events.to_sql("events", con = connect, if_exists = "replace", index = False)

1118352

# Données freeze frame de Skill Corner sur les centres

De même que pour les données event, nous disposons de 1 fichier par match pour ces données, nous allons donc concaténer l'ensemble des données de chaque fichier dans un seul jeu de données.

In [29]:
# Liste des noms des fichiers à ouvrir
liste_fichier_freeze_frames = os.listdir("Projet_centres_data/SKC_crosses_freeze_frames")

Cependant, cette fois nous souhaitons aussi conserver l'information "match_id" (Skill Corner) pour chaque match, qui n'est pas initialement pas présente dans les données "freeze frames".  
Cette information est en fait stockée dans le nom des fichiers qui sont de la forme "match_id.json".  
Pour ce faire, nous allons, grâce à une boucle for : 
- Parcourir la liste des noms des fichiers à lire (le nom correspond au "match_id" Skill Corner)
- Lire le fichier correspondant
- Ajouter l'information "match_id_SKC" dans une nouvelle colonne (le "match_id" sera ajouté à toutes les lignes du dataframe)
- Concaténer le jeu de données (dataframe) obtenu lors de l'itération au jeu de données comprenant l'ensemble des données des fichier lus lors des précédentes itérations.

In [30]:
# Création du dataframe qui contiendra le jeu de données concaténé (les données de l'ensemble des fichiers de données "freeze frames")
freeze_frames = pd.DataFrame()

# Boucle itérative
for fichier_freeze_frames in liste_fichier_freeze_frames :
    # Lecture du fichier correspondant
    freeze_frames_import = pd.read_json("Projet_centres_data/SKC_crosses_freeze_frames/" + fichier_freeze_frames)
    
    # Création de la colonne "match_id_SKC"
    # Nous enlevons la partie ".json" de la chaine de caractères correspondant au nom du fichier
    freeze_frames_import["match_id_SKC"] = int(fichier_freeze_frames.replace(".json", ""))

    # Concaténation du dataframe obtenu au dataframe comprenenant le jeu de données
    freeze_frames = pd.concat([freeze_frames, freeze_frames_import])

# De même que pour les données events, nous allons trier les freeze frames par ordre chronologique pour chaque match
freeze_frames.sort_values(by = ["match_id_SKC", "frame"], inplace = True)

# Pour les mêmes raisons que pour les données events, nous allons créer un nouvel index pour identifier les freeze frames.
freeze_frames.reset_index(drop = True, inplace = True)

# Nous supprimons la variable "image_corners_projection" qui nous est inutile
freeze_frames.drop("image_corners_projection", axis = 1, inplace = True)

In [31]:
# Aperçu des données
freeze_frames.head(2)

,data,possession,frame,timestamp,period,event_id,event_x,event_y,is_matched,match_id_SKC
0,"[{'track_id': 12235, 'trackable_object': 12235...","{'group': 'away team', 'trackable_object': None}",4820,2025-02-05 00:04:29.000,1.0,54cbad0c-1f05-4708-9b7f-2cadeae0636b,35.35,-27.71,True,1020089
1,"[{'track_id': 12235, 'trackable_object': 12235...","{'group': 'away team', 'trackable_object': 159...",4834,2025-02-05 00:04:30.400,1.0,28239a7e-4d65-4618-99f3-2763088e5e92,42.6125,-26.86,True,1020089


Il faut traiter la colonne "possession", qui contient des dictionnaires comprenant chacuns le joueur et l'équipe en possession du ballon au moment ou la frame a été capturée.  
En effet, ces dictionnaires ne peuvent pas être stocké par SQLite...  
Pour ce faire, nous allons procéder comme pour les listes de coordonnées précédentes :
- Supprimer et récupérer la colonne du dataframe
- Eclater cette colonne en 2 nouvelles colonnes qui contiennent respectivement l'équipe en possession du ballon et le joueur en possession du ballon lors de la frame

In [32]:
freeze_frames_possession = freeze_frames.pop("possession")
freeze_frames[["group", "tackable_object"]] = pd.json_normalize(freeze_frames_possession)

Nous allons maintenant traiter la colonne "data", qui contient pour chaque frame des informations (position, vitesse, etc) pour chaque joueur et le ballon.  
Ces informations sont les données de tracking.  
Pour chaque frame, la valeur de cette colonne "data" est en fait une liste de dictionnaire.  
Chaque dictionnaire de la liste correspond aux informations (position, vitesse, etc) pour un joueur ou le ballon.  
De ce fait, les données de cette colonne "data" ne peuvent pas être stockées par SQLite.  
Cependant, nous pouvons voir chaque élèment de cette colonne comme un dataframe, car ces élèments sont des listes de dictionnaires (possédant tous les même clés).  
De ce fait, en convertissant chaque ligne de cette colonne en un dataframe, nous obtiendrons donc (nombre de frames) dataframes.  

Pour effectuer cette manipulation, nous allons utilisé la commande "apply" de Pandas. Lorsque le paramètre de la commande "axis" = 1, cette méthode permet d'appliquer une fonction (passée en paramètre) à chaque ligne du dataframe.  
Grâce à cela, nous allons pouvoir, pour chaque frame, leur appliquer une fonction extrayant l'information "data" en la convertissant en dataframe, et ajouter à ce dataframe créé (pour chaque frame), l'information sur l'identifiant Stats Bomb de l'event et le match id Skill Corner.  
Cette fonction se nomme "transform_tracking_data", et est définie dans la cellule ci-dessous.  
La commande apply, appliquée au dataframe avec la fonction "transform_tracking_data", renverra une série de tous les dataframes créés, concaténés les uns en dessous des autres.  
Nous convertissons ensuite cette série de dataframe en liste de dataframe avec la commande "tolist", car chaque ligne de la série est en réalité un dataframe.  
Et finalement, nous appliquerons la fonction "concat" afin de concaténer l'ensemble des dataframes de la liste créée, les uns en dessous des autres.

In [33]:
def transform_tracking_data(row) :
    # Création du dataframe contenant l'information "data" pour la ligne itérée.
    # Cette information "data" est donc une liste de dictionnaire, qui peut être convertie en dataframe
    df = pd.DataFrame(row.data)
    # Ajout de l'information sur l'id de l'event
    df["event_id"] = row.event_id
    # Ajout de l'information sur le match id
    df["match_id_SKC"] = row.match_id_SKC
    return df

In [34]:
tracking_data = pd.concat((freeze_frames.apply(transform_tracking_data, axis = 1)).tolist())

# Suppression de la colonne data du dataframe initial qui contient les freeze frames
freeze_frames.drop("data", axis = 1, inplace = True)

In [35]:
# Ecriture des données importées dans une table de la BDD
freeze_frames.to_sql("freeze_frames", con = connect, if_exists = "replace", index = False)

20576

In [36]:
# Ecriture des données importées dans une table de la BDD
tracking_data.to_sql("tracking_data", con = connect, if_exists = "replace", index = False)

467914

# Données off ball runs de Skill Corner

Les données off ball runs de Skill Corner sont également agrégées par match, nous avons donc un fichier par match.  
Comme pour les freeze frames, le nom de ces fichiers est de la forme "match_id.json".  

In [37]:
# Liste des noms des fichiers à ouvrir
liste_fichier_off_ball_runs = os.listdir("Projet_centres_data/SKC_off_ball_runs")

In [38]:
# Création du dataframe contenant les données de l'ensemble des matchs à disposition.
# Comme précédemment, l'ensemble des données importées sont concaténées les unes en dessous des autres avec la commande "concat".
off_ball_runs = pd.concat([pd.read_json("Projet_centres_data/SKC_off_ball_runs/" + fichier_off_ball_runs)
                           for fichier_off_ball_runs in liste_fichier_off_ball_runs])

In [39]:
# Ecriture des données importées dans une table de la BDD
off_ball_runs.to_sql("off_ball_runs", con = connect, if_exists = "replace", index = False)

8489

# Données des cotés vers lesquels attaque les équipes

Nous allons maintenant importer les données qui nous permettent de savoir, pour chaque match id Skill Corner, vers quel coté attaque l'équipe à domicile en 1ère et 2nd période

In [40]:
# Importation du fichier
SKC_cote_attaque = pd.read_json("Projet_centres_data/data_add/SKC_matches_data.json")

In [41]:
# Aperçu des données
SKC_cote_attaque

,1547880,1547881,1547882,1547883,1547884,1547885,1547886,1547887,1547888,1536669,...,1034262,1024091,1023376,1022715,1023377,1028727,1022717,1021411,1020753,1020089
0,left_to_right,right_to_left,right_to_left,left_to_right,left_to_right,right_to_left,right_to_left,right_to_left,right_to_left,right_to_left,...,left_to_right,left_to_right,right_to_left,right_to_left,right_to_left,left_to_right,right_to_left,right_to_left,left_to_right,right_to_left
1,right_to_left,left_to_right,left_to_right,right_to_left,right_to_left,left_to_right,left_to_right,left_to_right,left_to_right,left_to_right,...,right_to_left,right_to_left,left_to_right,left_to_right,left_to_right,right_to_left,left_to_right,left_to_right,right_to_left,left_to_right


Les colonnes de ce dataframe correspondent à l'ensemble des matchs Skill Corner dont nous disposons, tandis que les lignes 0 et 1 contiennent respectivement le coté d'attaque de l'équipe à domicile en 1ère et 2nd période.  
Nous allons donc transposer ce dataframe, renommer le nom de ses colonnes et modifier son nom afin de le rendre plus intuitif

In [42]:
SKC_cote_attaque_home_team = SKC_cote_attaque.transpose().rename({0 : "premiere_periode", 1 : "seconde_periode"}, axis = 1)

In [43]:
# Aperçu des données modifiées
SKC_cote_attaque_home_team.head(2)

,premiere_periode,seconde_periode
1547880,left_to_right,right_to_left
1547881,right_to_left,left_to_right


In [44]:
# Ecriture des données importées dans une table de la BDD
# De plus, nous allons mettre le paramètre "index = True", qui nous permet de garder l'information des match id SKC,
# qui sont les index du dataframe
SKC_cote_attaque_home_team.to_sql("SKC_cote_attaque_home_team", con = connect, if_exists = "replace", index = True, index_label = "match_id_SKC")

306

# Données Lineup de Stats Bomb

Enfin, nous allons importer les données lineup de Stats Bomb qui sont elles aussi agrégées par match.

In [45]:
# Liste des noms des fichiers à ouvrir
liste_fichier_lineups = os.listdir("Projet_centres_data/SB_lineups")

Cependant, de même que pour les données freeze frames, nous ne disposons pas directement de l'information sur les match id Stats Bomb.  
De ce fait, il est nécessaire, à chaque fois qu'on import les données lineup d'un match, d'ajouter une colonne contenant le match id à ces données.  
Nous pourrons récupérer le match id correspondant au fichier importé car le match id est contenu dans le nom des fichiers.

In [46]:
exemple_lineup = pd.read_json("Projet_centres_data/SB_lineups/3894037.json")
print("Colonne du dataframe :", exemple_lineup.columns)
print("Index du dataframe :", exemple_lineup.index)

Colonne du dataframe : Index([147, 156], dtype='int64')
Index du dataframe : Index(['team_id', 'lineup', 'formations', 'events'], dtype='object')


De plus, comme nous pouvons le voir, les données de ces fichiers lineup sont "inversées" : les variables sont contenues dans les lignes et les individus (les team id) sont contenus dans les colonnes.  
Cela n'est d'une part, pas cohérent, et d'autre part, posera posera problème lorsque nous vondront concaténer l'ensemble des fichiers importés.  
Pour cela, il est nécessaire, à chaque importation de fichier, de "transposer" les dataframe résultant de l'importation du fichier.  
Cela permettra de définir les team id en tant que lignes et les variables en tant que colonnes.

Enfin, nous ne garderons que les colonnes "team_id" et "lineup", car les autres informations ne nous sont pas utiles.

In [47]:
# Création du dataframe qui contiendra l'ensemble des données, concaténées
lineups = pd.DataFrame()

# Boucle pour traverser l'ensemble des fichiers
for fichier_lineups in liste_fichier_lineups :
    # Lecture du fichier correspondant et transposition des colonnes/lignes
    lineups_import = pd.read_json("Projet_centres_data/SB_lineups/" + fichier_lineups).transpose()

    # On ne garde que les colonnes "team_id" et "lineup"
    lineups_import = lineups_import[["team_id", "lineup"]]
    
    # Création de la colonne "match_id_SB"
    # Nous enlevons la partie ".json" de la chaine de caractères correspondant au nom du fichier
    lineups_import["match_id_SB"] = int(fichier_lineups.replace(".json", ""))

    # Concaténation du dataframe obtenu au dataframe comprenenant le jeu de données
    lineups = pd.concat([lineups, lineups_import])

In [48]:
# Aperçu des données
lineups.head(2)

,team_id,lineup,match_id_SB
147,147,"[{'player_id': 2941, 'birth_date': '1998-02-25...",3894037
156,156,"[{'player_id': 7378, 'birth_date': '1987-09-28...",3894037


Avant d'écrire ces données dans la BDD SQLite, il est important de modifier la colonne "lineup" qui contient, comme pour la colonne "data" des freeze frames, des listes de dictionnaire.  
En effet, chaque liste de dictionnaire contient, pour une équipe et un match donnés, des informations sur l'ensemble des joueurs de cette équipe inscrits sur la feuille de match.  
Nous pouvons donc, pour chaque ligne de la colonne lineup, convertir la liste de dictionnaire en dataframe et concaténer l'ensemble des dataframes résultants.  
Nous allons encore une fois utiliser la commande apply de Pandas.

In [49]:
def transform_lineups(row) :
    # Création du dataframe correspondant à la liste de dictionnaire de la colonne "lineup" de la ligne itérée.
    # Dataframe qui contient donc dans les lignes, l'ensemble des joueurs d'une équipe inscrits sur la feuille de match
    # et dans les colonnes, les informations sur ces joueurs
    df = pd.DataFrame(row.lineup)
    # Ajout de l'information "team_id"
    df["team_id"] = row.team_id
    # Ajout de l'information sur les match id
    df["match_id_SB"] = row.match_id_SB
    return df

In [50]:
# On modifie le dataframe initial avec la commande apply, de la même façon que pour les freeze frames
# Le dataframe résultant contiendra donc sur les lignes, les informations sur l'ensemble des joueurs de toutes les équipes, pour tous les matchs.
# Il contiendra aussi les colonnes "team_id" et "match_id_SB" qui ont été ajoutées par la fonction "transform_lineups"
lineups = pd.concat((lineups.apply(transform_lineups, axis = 1)).tolist())

Ce dataframe résultant contient également des colonnes composées de dictionnaire ("country", "positions" et "stats").  
Cependant, ces informations ne nous sont pas utiles, nous avons donc juste à les supprimer du dataframe.

In [51]:
lineups.drop(["country", "positions", "stats"], axis = 1, inplace = True)

In [52]:
# Aperçu des données
lineups.head(2)

,player_id,birth_date,player_gender,player_height,player_weight,jersey_number,team_id,match_id_SB
0,2941,1998-02-25,male,185.0,76.0,23,147,3894037
1,3044,1994-08-08,male,182.0,81.0,99,147,3894037


In [53]:
# Renommage des colonnes pour indiquer le fait que les id sont des id Stats Bomb
dico_rename = {"player_id" : "player_id_SB", "team_id" : "team_id_SB"}
lineups.rename(dico_rename, axis = 1, inplace = True)

In [54]:
# Ecriture des données importées dans une table de la BDD
lineups.to_sql("lineups", con = connect, if_exists = "replace", index = False)

12227

# Fermeture de la base de données

In [55]:
# Fin de l'importation des données, nous fermons donc le connecteur à la base de données.
connect.close()